In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense , Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_squared_error


C:\Users\za220\AppData\Roaming\Python\Python313\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/attr_value.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
C:\Users\za220\AppData\Roaming\Python\Python313\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/tensor.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
C:\Users\za220\AppData\Roaming\Python\Python313\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/resource_handle.proto. Please 

In [2]:
start_date="2015-9-16"

In [3]:
end_date=datetime.now()

In [4]:
symbol=['AAPL']

In [5]:
AAPL=yf.download(symbol , start_date , end_date, multi_level_index=False).Close
AAPL

C:\Users\za220\AppData\Local\Temp\ipykernel_18536\2694260390.py:1: FutureWarning: YF.download() has changed argument auto_adjust default to True
  AAPL=yf.download(symbol , start_date , end_date, multi_level_index=False).Close
[*********************100%***********************]  1 of 1 completed


Date
2015-09-16     26.160309
2015-09-17     25.600733
2015-09-18     25.495119
2015-09-21     25.890625
2015-09-22     25.483875
                 ...    
2025-09-19    245.500000
2025-09-22    256.079987
2025-09-23    254.429993
2025-09-24    252.309998
2025-09-25    256.869995
Name: Close, Length: 2522, dtype: float64

In [6]:
# Step-2:
#Intilazing MinMaxScaler to normalize the data between 0 and 1
scaler=MinMaxScaler(feature_range=(0 , 1))

data=AAPL.values.reshape(-1 , 1)
# Scaled the data for training 
scaled_data=scaler.fit_transform(data)
scaled_data

array([[0.02331257],
       [0.02095625],
       [0.02051153],
       ...,
       [0.98453031],
       [0.97560325],
       [0.99480489]], shape=(2522, 1))

In [7]:
 # Step-3 
# Defining training data length as 80% of the total data
training_data_len=int(len(scaled_data) * 0.8)

In [26]:
# Split the data into training set 
training_data=scaled_data[: training_data_len]
testing_data=scaled_data[training_data_len:]

In [27]:
training_data

array([[0.02331257],
       [0.02095625],
       [0.02051153],
       ...,
       [0.65529669],
       [0.65988378],
       [0.64495504]], shape=(2017, 1))

In [10]:
# Creating empty list for features (x_train and y_train) 
x_train=[]
y_train=[]

In [11]:
window_size=60

In [12]:
for i in range(60 , len(training_data)):
    x_train.append(training_data[i-60:i , 0])
    y_train.append(training_data[i , 0])

In [73]:
x_test=[]
y_test=[]
for i in range(60, len(testing_data)):
    x_test.append(testing_data[i-60:i , 0])
    y_test.append(testing_data[i,0])

In [74]:
# convert list to numpy _ array t0 model training 
x_train=np.array(x_train)
y_train=np.array(y_train)
x_test=np.array(x_test)
y_test=np.array(y_test)

In [76]:
# Convert it into 3d sample if we want to train lstm model convert it to the 3d sample
x_train=np.reshape(x_train ,(x_train.shape[0] , x_train.shape[1] , 1))
x_test=np.reshape(x_test , (x_test.shape[0] ,x_test.shape[1] , 1)) # re shape the x_test 

In [77]:
print(x_train.shape) # u change the dimension of vector from 2d to 3d for modelling
print(x_test.shape)

(1957, 60, 1)
(445, 60, 1)


In [16]:
model=Sequential()

In [17]:
# First lstm layer with 50 units and return_sequence
model.add(LSTM(units=200 , return_sequences=True, input_shape=(x_train.shape[1] , 1)))
model.add(Dropout(0.2)) # Dropout layer to prevent overfitting

C:\Users\za220\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [18]:
#Second layer of lstm layer 
model.add(LSTM(units=50 , return_sequences=False))
model.add(Dropout(0.2))

In [19]:
#Dense layer with 25 units 
model.add(Dense(units=25))

In [20]:
#Output layer with 1 units (Predicted price)
model.add(Dense(units=1))

In [21]:
#Compile the model using adam optimizer and mean squared error as loss function
model.compile(optimizer='adam' , loss='mean_squared_error')

In [22]:
# Save model after stopping
import tensorflow as tf
tf.compat.v1.reset_default_graph()

In [23]:
model.fit(x_train , y_train , batch_size=10 , epochs=10)

Epoch 1/10
196/196 ━━━━━━━━━━━━━━━━━━━━ 39s 127ms/step - loss: 0.0035
Epoch 2/10
196/196 ━━━━━━━━━━━━━━━━━━━━ 43s 140ms/step - loss: 0.0011    
Epoch 3/10
196/196 ━━━━━━━━━━━━━━━━━━━━ 38s 126ms/step - loss: 7.8072e-04
Epoch 4/10
196/196 ━━━━━━━━━━━━━━━━━━━━ 25s 125ms/step - loss: 7.0157e-04
Epoch 5/10
196/196 ━━━━━━━━━━━━━━━━━━━━ 25s 128ms/step - loss: 6.1202e-04
Epoch 6/10
196/196 ━━━━━━━━━━━━━━━━━━━━ 26s 134ms/step - loss: 7.0026e-04
Epoch 7/10
196/196 ━━━━━━━━━━━━━━━━━━━━ 27s 135ms/step - loss: 7.2510e-04
Epoch 8/10
196/196 ━━━━━━━━━━━━━━━━━━━━ 25s 125ms/step - loss: 5.7521e-04
Epoch 9/10
196/196 ━━━━━━━━━━━━━━━━━━━━ 26s 130ms/step - loss: 6.3864e-04
Epoch 10/10
196/196 ━━━━━━━━━━━━━━━━━━━━ 25s 129ms/step - loss: 5.8588e-04


In [83]:
y_pred=model.predict(x_test)
y_pred=scaler.inverse_transform(y_pred)
y_test_inv=scaler.inverse_transform(y_test.reshape(-1 , 1))

14/14 ━━━━━━━━━━━━━━━━━━━━ 3s 113ms/step


In [84]:
future_input=testing_data[-60:]
future_input=future_input.reshape(1, 60, 1)
future_prediction=[]

In [85]:
for _ in range(30):
    pred=model.predict(future_input)
    future_prediction.append(pred[0 ,0])
    future_input = np.concatenate((future_input[:,1:,:], pred.reshape(1,1,1)), axis=1)
    

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 133ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 132ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 143ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 118ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 129ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 126ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 135ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 124ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 124ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 124ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 128ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 130ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 128ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0

In [86]:
future_prediction = np.array(future_prediction).reshape(-1,1)
future_prices = scaler.inverse_transform(future_prediction)
print(future_prices)

[[244.96867]
 [245.44928]
 [244.6095 ]
 [242.9802 ]
 [240.94797]
 [238.76321]
 [236.57101]
 [234.444  ]
 [232.41026]
 [230.47354]
 [228.62651]
 [226.85846]
 [225.15909]
 [223.52016]
 [221.93556]
 [220.40105]
 [218.91364]
 [217.47115]
 [216.07199]
 [214.7147 ]
 [213.39787]
 [212.12016]
 [210.88022]
 [209.67674]
 [208.50838]
 [207.37387]
 [206.27193]
 [205.20146]
 [204.1612 ]
 [203.1501 ]]
